# 顧客レビューを論点分解
- 要件：サーバレスコンピュート or DBR 16.0 以降

注意：  
- **日本語訳データをLLMで論点ごとに抽出＆要約しますが、15分程度かかります。**  
- **省略したい場合、`No.2　顧客レビューから論点抽出してサマライズ`はスキップ**できます（GithubからCSVダウンロードして代替します）  
- **No.2　をスキップする場合、`No.3. 論点ごとのサマリをポジ/ネガごとにレコード展開`に進んでください。**

#### 処理概要
1. LLMを使ってでレビューから論点抽出＆要約
1. 論点ごとに抽出された要約文章をレコード展開
1. ダッシュボードで可視化

In [0]:
%run ./00_config

## 1. データ準備
日本語翻訳済みのCSVをVolumeから読み込みます

In [0]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, DateType,
    BooleanType, LongType
)

# ---------------- Schema 定義 ----------------
schema = StructType([
    StructField("review_id", LongType(), True),
    StructField("user_id", LongType(), True),
    StructField("review_date", DateType(), True),
    StructField("review", StringType(), True),
    StructField("flight_id", StringType(), True),
    StructField("flight_date", DateType(), True),
    StructField("route_id", StringType(), True),
    StructField("seat_type", StringType(), True),
    StructField("is_recommended", BooleanType(), True),
    StructField("rating_overall", IntegerType(), True),
    StructField("rating_seat_comfort", IntegerType(), True),
    StructField("rating_cabin_staff_service", IntegerType(), True),
    StructField("rating_ground_service", IntegerType(), True),
    StructField("rating_value_for_money", IntegerType(), True),
    StructField("rating_food_beverages", IntegerType(), True),
    StructField("rating_ife", IntegerType(), True),
    StructField("rating_wifi_connectivity", IntegerType(), True)
])

# -------------- CSV 読み込み ------------------
csv_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/reviews.csv"  # ファイル名まで
df = (
    spark.read.format("csv")
         .option("header", "true")
         .option("quote", '"')
         .option("escape", '"')
         .option("multiLine", "true")
         .schema(schema)           # ★ スキーマを適用
         .load(csv_path)
)

# -------------- テーブル保存 -------------------
df.write.format("delta").mode("overwrite").saveAsTable(
    f"{MY_CATALOG}.{MY_SCHEMA}.bz_reviews"
)

print(df.count())
print(df.columns)   # 型確認
display(df.limit(100))

## 2. 顧客レビューから論点抽出してサマライズ
作成テーブル：`sv_reviews_summaries_by_topics`  
処理：
今回扱う顧客レビューは設問設計がなされていない自由記述であり、複数要素が混在する。  
このままでは分析に使いにくいので、感情 x 論点ごとに要素分解する。  
分解要素は次のとおり。

- 感情
  - ポジティブ
  - ネガティブ
- 論点
  - 食事と飲み物に対する評価
  - 機内エンターテイメントに対する評価
  - 機内Wi-Fiと接続性に対する評価
  - 価格に対する価値の評価
  - 地上サービスに対する評価
  - 客室スタッフのサービスに対する評価
  - 座席の快適さに対する評価
  - その他


下記セル8-9は15分程度かかります。この処理をスキップしたい場合、`3. 論点ごとのサマリをポジ/ネガごとにレコード展開`にショートカット可能です

In [0]:
import time

# タイマークラス
class TimeMeasurement:
    def __init__(self):
        self.start_time = None
        self.end_time = None

    def start(self):
        """タイマーを開始します。"""
        self.start_time = time.time()

    def stop(self):
        """タイマーを停止します。"""
        self.end_time = time.time()

    def elapsed_time(self):
        """経過時間を秒単位で返します。"""
        if self.start_time is None or self.end_time is None:
            raise ValueError("タイマーが開始または停止されていません。")
        return self.end_time - self.start_time

    def elapsed_time_formatted(self):
        """経過時間を HH:MM:SS 形式で返します。"""
        return time.strftime("%H:%M:%S", time.gmtime(self.elapsed_time()))

In [0]:
# 既存テーブル削除
spark.sql(f"DROP TABLE IF EXISTS {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# タイマー
timer = TimeMeasurement()

MAX_RECORDS = 1000      # 全体の処理件数
BATCH_SIZE = 10         # バッチサイズ

# 主キーの最小値と最大値を取得
min_max = spark.sql(f"""
SELECT
  MIN(review_id) AS min_no,
  MAX(review_id) AS max_no
FROM
  {MY_CATALOG}.{MY_SCHEMA}.bz_reviews
""").collect()[0]

min_no = int(min_max["min_no"])
max_no = int(min_max["max_no"])

# 処理件数が最大値を超えないように調整
if MAX_RECORDS > (max_no - min_no + 1):
    MAX_RECORDS = max_no - min_no + 1

# 範囲指定を使用したバッチ処理
for start_no in range(min_no, min_no + MAX_RECORDS, BATCH_SIZE):
    end_no = start_no + BATCH_SIZE - 1

    # タイマー開始
    timer = TimeMeasurement()
    timer.start()

    # プロンプト生成とAIクエリ実行
    batch_df = spark.sql(f"""
        WITH prompts AS (
          SELECT
            review_id,
            user_id,
            review_date,
            review,

            ------------------------------
            -- 1. ポジティブ
            ------------------------------
            -- 1-1. ポジティブ_食事と飲み物
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「食事と飲み物」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「食事と飲み物」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_food_and_beverages_prompt,

            -- 1-2. ポジティブ_機内エンターテイメント
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「機内エンターテイメント」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内エンターテイメント」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_inflight_entertainment_prompt,

            -- 1-3. ポジティブ_機内Wi-Fiと接続性
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「機内Wi-Fiと接続性」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内Wi-Fiと接続性」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_wifi_connectivity_prompt,

            -- 1-4. ポジティブ_価格に対する価値
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「価格に対する価値」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「価格に対する価値」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_value_for_money_prompt,

            -- 1-5. ポジティブ_地上サービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「地上サービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「地上サービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_ground_service_prompt,

            -- 1-6. ポジティブ_客室スタッフのサービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「客室スタッフのサービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「客室スタッフのサービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_cabin_staff_service_prompt,

            -- 1-7. ポジティブ_座席の快適さ
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ポジティブ、かつ「座席の快適さ」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「座席の快適さ」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS pos_seat_comfort_prompt,

            ------------------------------
            -- 2. ネガティブ
            ------------------------------
            -- 2-1. ネガティブ_食事と飲み物
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「食事と飲み物」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「食事と飲み物」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_food_and_beverages_prompt,

            -- 2-2. ネガティブ_機内エンターテイメント
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「機内エンターテイメント」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内エンターテイメント」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_inflight_entertainment_prompt,

            -- 2-3. ネガティブ_機内Wi-Fiと接続性
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「機内Wi-Fiと接続性」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「機内Wi-Fiと接続性」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_wifi_connectivity_prompt,

            -- 2-4. ネガティブ_価格に対する価値
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「価格に対する価値」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「価格に対する価値」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_value_for_money_prompt,

            -- 2-5. ネガティブ_地上サービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「地上サービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「地上サービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_ground_service_prompt,

            -- 2-6. ネガティブ_客室スタッフのサービス
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「客室スタッフのサービス」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「客室スタッフのサービス」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_cabin_staff_service_prompt,

            -- 2-7. ネガティブ_座席
            CONCAT(
              "[指示]次の顧客レビューから[抽出条件]に該当する内容を抽出し、明快で簡潔な一つの文章に要約して下さい",
              "\n[顧客レビュー]",
              review,
              "\n[抽出条件]ネガティブ、かつ「座席の快適さ」に該当する内容",
              "\n[厳守事項]",
              "\n * 該当がなければ、必ず「-」とだけ回答してください",
              "\n * 要約結果のみ出力してください。質問からすでに明らかな文脈は含めない（例えば「座席の快適さ」など自明のものは外すこと）",
              "\n * 顧客本人が書いたレビュー風の表現にしてください（例えば「〜についての記載はありません」「〜について触れられていません」など、分析用に補正したとわかる表現は避けること）",
              "\n * 背景や評価理由を省略し過ぎず、サービス向上の具体的なアクションに繋がるポイントを明確にしてください"
            ) AS neg_seat_comfort_prompt,

            flight_id,
            flight_date,
            route_id,
            seat_type,
            is_recommended,
            rating_overall,
            rating_seat_comfort,
            rating_cabin_staff_service,
            rating_ground_service,
            rating_value_for_money,
            rating_food_beverages,
            rating_ife,
            rating_wifi_connectivity
          FROM {MY_CATALOG}.{MY_SCHEMA}.bz_reviews
          WHERE review_id BETWEEN {start_no} AND {end_no}
          ORDER BY review_id
        ),
        summaries_by_topics AS (
          SELECT
            review_id,
            user_id,
            review_date,
            review,

            ------------------------------
            -- 1. ポジティブ
            ------------------------------
            -- 1-1. ポジティブ_食事と飲み物に対する評価
            pos_food_and_beverages_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_food_and_beverages_prompt, failOnError => False).result AS pos_food_and_beverages_result,

            -- 1-2. ポジティブ_機内エンターテイメントに対する評価
            pos_inflight_entertainment_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_inflight_entertainment_prompt, failOnError => False).result AS pos_inflight_entertainment_result,

            -- 1-3. ポジティブ_機内Wi-Fiと接続性に対する評価
            pos_wifi_connectivity_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_wifi_connectivity_prompt, failOnError => False).result AS pos_wifi_connectivity_result,

            -- 1-4. ポジティブ_価格に対する価値の評価
            pos_value_for_money_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_value_for_money_prompt, failOnError => False).result AS pos_value_for_money_result,

            -- 1-5. ポジティブ_地上サービスに対する評価
            pos_ground_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_ground_service_prompt, failOnError => False).result AS pos_ground_service_result,

            -- 1-6. ポジティブ_客室スタッフのサービスに対する評価
            pos_cabin_staff_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_cabin_staff_service_prompt, failOnError => False).result AS pos_cabin_staff_service_result,

            -- 1-7. ポジティブ_座席の快適さに対する評価
            pos_seat_comfort_prompt,
            ai_query('databricks-claude-3-7-sonnet', pos_seat_comfort_prompt, failOnError => False).result AS pos_seat_comfort_result,

            ------------------------------
            -- 2. ネガティブ
            ------------------------------
            -- 2-1. ネガティブ_食事と飲み物に対する評価
            neg_food_and_beverages_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_food_and_beverages_prompt, failOnError => False).result AS neg_food_and_beverages_result,

            -- 2-2. ネガティブ_機内エンターテイメントに対する評価
            neg_inflight_entertainment_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_inflight_entertainment_prompt, failOnError => False).result AS neg_inflight_entertainment_result,

            -- 2-3. ネガティブ_機内Wi-Fiと接続性
            neg_wifi_connectivity_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_wifi_connectivity_prompt, failOnError => False).result AS neg_wifi_connectivity_result,

            -- 2-4. ネガティブ_価格に対する価値の評価
            neg_value_for_money_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_value_for_money_prompt, failOnError => False).result AS neg_value_for_money_result,

            -- 2-5. ネガティブ_地上サービスに対する評価
            neg_ground_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_ground_service_prompt, failOnError => False).result AS neg_ground_service_result,

            -- 2-6. ネガティブ_客室スタッフのサービスに対する評価
            neg_cabin_staff_service_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_cabin_staff_service_prompt, failOnError => False).result AS neg_cabin_staff_service_result,

            -- 2-7. ネガティブ_座席の快適さに対する評価
            neg_seat_comfort_prompt,
            ai_query('databricks-claude-3-7-sonnet', neg_seat_comfort_prompt, failOnError => False).result AS neg_seat_comfort_result,

            flight_id,
            flight_date,
            route_id,
            seat_type,
            is_recommended,
            rating_overall,
            rating_seat_comfort,
            rating_cabin_staff_service,
            rating_ground_service,
            rating_value_for_money,
            rating_food_beverages,
            rating_ife,
            rating_wifi_connectivity
          FROM prompts
        )
        SELECT
          review_id,
          user_id,
          review_date,
          review,
          
          ------------------------------
          -- 1. ポジティブ
          ------------------------------
          -- 1-1. ポジティブ_食事と飲み物
          -- pos_food_and_beverages_prompt,
          COALESCE(pos_food_and_beverages_result, '-') AS pos_food_and_beverages_summary,

          -- 1-2. ポジティブ_機内エンターテイメント
          -- pos_inflight_entertainment_prompt,
          COALESCE(pos_inflight_entertainment_result, '-') AS pos_inflight_entertainment_summary,

          -- 1-3. ポジティブ_機内Wi-Fiと接続性
          -- pos_wifi_connectivity_prompt,
          COALESCE(pos_wifi_connectivity_result, '-') AS pos_wifi_connectivity_summary,

          -- 1-4. ポジティブ_価格に対する価値
          -- pos_value_for_money_prompt,
          COALESCE(pos_value_for_money_result, '-') AS pos_value_for_money_summary,

          -- 1-5. ポジティブ_地上サービス
          -- pos_ground_service_prompt,
          COALESCE(pos_ground_service_result, '-') AS pos_ground_service_summary,

          -- 1-6. ポジティブ_客室スタッフのサービス
          -- pos_cabin_staff_service_prompt,
          COALESCE(pos_cabin_staff_service_result, '-') AS pos_cabin_staff_service_summary,

          -- 1-7. ポジティブ_座席の快適さ
          -- pos_seat_comfort_prompt,
          COALESCE(pos_seat_comfort_result, '-') AS pos_seat_comfort_summary,

          ------------------------------
          -- 2. ネガティブ
          ------------------------------
          -- 2-1. ネガティブ_食事と飲み物
          -- neg_food_and_beverages_prompt,
          COALESCE(neg_food_and_beverages_result, '-') AS neg_food_and_beverages_summary,

          -- 2-2. ネガティブ_機内エンターテイメント
          -- neg_inflight_entertainment_prompt,
          COALESCE(neg_food_and_beverages_result, '-') AS neg_inflight_entertainment_summary,

          -- 2-3. ネガティブ_機内Wi-Fiと接続性
          -- neg_wifi_connectivity_prompt,
          COALESCE(neg_wifi_connectivity_result, '-') AS neg_wifi_connectivity_summary,

          -- 2-4. ネガティブ_価格に対する価値
          -- neg_value_for_money_prompt,
          COALESCE(neg_value_for_money_result, '-') AS neg_value_for_money_summary,

          -- 2-5. ネガティブ_地上サービス
          -- neg_ground_service_prompt,
          COALESCE(neg_ground_service_result, '-') AS neg_ground_service_summary,

          -- 2-6. ネガティブ_客室スタッフのサービス
          -- neg_cabin_staff_service_prompt,
          COALESCE(neg_cabin_staff_service_result, '-') AS neg_cabin_staff_service_summary,

          -- 2-7. ネガティブ_座席の快適さ
          -- neg_seat_comfort_prompt,
          COALESCE(neg_seat_comfort_result, '-') AS neg_seat_comfort_summary,

          flight_id,
          flight_date,
          route_id,
          seat_type,
          is_recommended,
          rating_overall,
          rating_seat_comfort,
          rating_cabin_staff_service,
          rating_ground_service,
          rating_value_for_money,
          rating_food_beverages,
          rating_ife,
          rating_wifi_connectivity
        FROM summaries_by_topics
    """)

    # テーブルにインクリメンタル書き込み
    (batch_df.write.format("delta")
        .mode("append")
        .option("mergeSchema", "true")
        .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics"))

    # タイマー停止
    timer.stop()

    # 所要時間の表示
    elapsed_seconds = timer.elapsed_time()
    elapsed_formatted = timer.elapsed_time_formatted()

    print(f"Processed range: {start_no + 1} - {end_no + 1} （所要時間: {elapsed_formatted} / {elapsed_seconds:.2f} 秒）")

In [0]:

# '''
# クレンジング
# llmが生成したサマリ文章に「-」意外に余計な文字列が含まれている場合、「-」に置き換える
# '''
# # 1-1. ポジティブ_食事と飲み物
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_food_and_beverages_summary = '-'
# WHERE  pos_food_and_beverages_summary LIKE '% -%';
# """)

# # 1-2. ポジティブ_機内エンターテイメント
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_inflight_entertainment_summary = '-'
# WHERE  pos_inflight_entertainment_summary LIKE '% -%';
# """)

# # 1-3. ポジティブ_機内Wi-Fiと接続性
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_wifi_connectivity_summary = '-'
# WHERE  pos_wifi_connectivity_summary LIKE '% -%';
# """)

# # 1-4. ポジティブ_価格に対する価値
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_value_for_money_summary = '-'
# WHERE  pos_value_for_money_summary LIKE '% -%';
# """)

# # 1-5. ポジティブ_地上サービス
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_ground_service_summary = '-'
# WHERE  pos_ground_service_summary LIKE '% -%';
# """)

# # 1-6. ポジティブ_客室スタッフのサービス
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_cabin_staff_service_summary = '-'
# WHERE  pos_cabin_staff_service_summary LIKE '% -%';
# """)

# # 1-7. ポジティブ_座席の快適さ
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    pos_seat_comfort_summary = '-'
# WHERE  pos_seat_comfort_summary LIKE '% -%';
# """)

# # 2-1. ネガティブ_食事と飲み物
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_food_and_beverages_summary = '-'
# WHERE  neg_food_and_beverages_summary LIKE '% -%';
# """)

# # 2-2. ネガティブ_機内エンターテイメント
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_inflight_entertainment_summary = '-'
# WHERE  neg_inflight_entertainment_summary LIKE '% -%';
# """)

# # 2-3. ネガティブ_機内Wi-Fiと接続性
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_wifi_connectivity_summary = '-'
# WHERE  neg_wifi_connectivity_summary LIKE '% -%';
# """)

# # 2-4. ネガティブ_価格に対する価値
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_value_for_money_summary = '-'
# WHERE  neg_value_for_money_summary LIKE '% -%';
# """)

# # 2-5. ネガティブ_地上サービス
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_ground_service_summary = '-'
# WHERE  neg_ground_service_summary LIKE '% -%';
# """)

# # 2-6. ネガティブ_客室スタッフのサービス
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_cabin_staff_service_summary = '-'
# WHERE  neg_cabin_staff_service_summary LIKE '% -%';
# """)

# # 2-7. ネガティブ_座席の快適さ
# spark.sql(f"""
# UPDATE {MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics
# SET    neg_seat_comfort_summary = '-'
# WHERE  neg_seat_comfort_summary LIKE '% -%';
# """)

In [0]:
# # ---------- CSV 出力 ----------
# t_df = spark.table(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME_TMP}/reviews_summaries_by_topics.csv"
# t_df.coalesce(1).toPandas().to_csv(out_path, index=False)
# print(out_path)

In [0]:
df = spark.table(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")
display(
  df.select(
    'review_id',
    'review_date',
    'pos_food_and_beverages_summary',
    'pos_inflight_entertainment_summary',
    'pos_wifi_connectivity_summary',
    'pos_value_for_money_summary',
    'pos_ground_service_summary',
    'pos_cabin_staff_service_summary',
    'pos_seat_comfort_summary',
    'neg_food_and_beverages_summary',
    'neg_inflight_entertainment_summary',
    'neg_wifi_connectivity_summary',
    'neg_value_for_money_summary',
    'neg_ground_service_summary',
    'neg_cabin_staff_service_summary',
    'neg_seat_comfort_summary'
  )
)

## 3. 論点ごとのサマリをポジ/ネガごとにレコード展開
作成テーブル：`gd_reviews_exstracted_by_categories_all`
処理：以下の論点ごとに横方向（カラム）に分かれているが、このままだと使いにくいので、縦方向（レコード）に展開する。  
処理のイメージ
```
会員ID  | 機内スタッフの接客の質     | 食事や飲み物の品質      | 座席の快適さ      | 地上サービスの質      | 価格に対する価値
A001   | *****                   | *****               | *****           | *****               | *****
A002   | *****                   | *****               | *****           | *****               | *****

↓

A001   | 機内スタッフのサービス| *****
A001   | 食事や飲み物        | *****
A001   | 座席の快適さ        | *****
A001   | 地上サービス        | *****
A001   | 価格に対する価値     | *****
A001   | その他             | *****
A002   | 機内スタッフのサービス| *****
A002   | 食事や飲み物        | *****
A002   | 座席の快適さ        | *****
A002   | 地上サービス        | *****
A002   | 価格に対する価値     | *****
A002   | その他             | *****
```

**【チート処理】**
`2. 顧客レビューから論点抽出してサマライズ`をスキップした場合、次のセルのコメントアウトを外して実行してください。

本来はLLMで処理してテーブル`sv_reviews_summaries_by_topics`を作りますが、  
代わりにVolumeからCSVデータを読み込んでテーブル作成することで、LLM処理の待機時間をショートカットします。

In [0]:
# from pyspark.sql.types import (
#     StructType, StructField, StringType, IntegerType, DateType,
#     BooleanType, LongType
# )

# # ---------------- Schema 定義 ----------------
# schema = StructType([
#     StructField("review_id", LongType(), True),
#     StructField("user_id", LongType(), True),
#     StructField("review_date", DateType(), True),
#     StructField("review", StringType(), True),
#     StructField("pos_food_and_beverages_summary", StringType(), True),
#     StructField("pos_inflight_entertainment_summary", StringType(), True),
#     StructField("pos_wifi_connectivity_summary", StringType(), True),
#     StructField("pos_value_for_money_summary", StringType(), True),
#     StructField("pos_ground_service_summary", StringType(), True),
#     StructField("pos_cabin_staff_service_summary", StringType(), True),
#     StructField("pos_seat_comfort_summary", StringType(), True),
#     StructField("neg_food_and_beverages_summary", StringType(), True),
#     StructField("neg_inflight_entertainment_summary", StringType(), True),
#     StructField("neg_wifi_connectivity_summary", StringType(), True),
#     StructField("neg_value_for_money_summary", StringType(), True),
#     StructField("neg_ground_service_summary", StringType(), True),
#     StructField("neg_cabin_staff_service_summary", StringType(), True),
#     StructField("neg_seat_comfort_summary", StringType(), True),
#     StructField("flight_id", StringType(), True),
#     StructField("flight_date", DateType(), True),
#     StructField("route_id", StringType(), True),
#     StructField("seat_type", StringType(), True),
#     StructField("is_recommended", BooleanType(), True),
#     StructField("rating_overall", IntegerType(), True),
#     StructField("rating_seat_comfort", IntegerType(), True),
#     StructField("rating_cabin_staff_service", IntegerType(), True),
#     StructField("rating_ground_service", IntegerType(), True),
#     StructField("rating_value_for_money", IntegerType(), True),
#     StructField("rating_food_beverages", IntegerType(), True),
#     StructField("rating_ife", IntegerType(), True),
#     StructField("rating_wifi_connectivity", IntegerType(), True)
# ])

# # -------------- CSV 読み込み ------------------
# df = spark.read.format("csv")\
#       .option("header", True) \
#       .option("multiLine", True) \
#       .option("quote", '"') \
#       .option("escape", '"') \
#       .schema(schema) \
#       .load(f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME_TMP}/reviews_summaries_by_topics.csv")

# # -------------- テーブル保存 -------------------
# df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# print(df.count())
# print(df.columns)
# display(df.limit(100))

In [0]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, DateType, IntegerType, BooleanType
from pyspark.sql.functions import explode, struct, lit, col, array

# データフレームの読み込み
df = spark.read.table(f"{MY_CATALOG}.{MY_SCHEMA}.sv_reviews_summaries_by_topics")

# スキーマ定義
schema = StructType([
    StructField("review_id", LongType(), True),                         # レビューID
    StructField("user_id", LongType(), True),                           # 会員ID
    StructField("review_date", DateType(), True),                       # レビュー投稿日
    StructField("review", StringType(), True),                          # レビュー本文
    StructField("flight_id", StringType(), True),                       # 航空機ID
    StructField("flight_date", DateType(), True),                       # 出発日
    StructField("route_id", StringType(), True),                        # 区間
    StructField("seat_type", StringType(), True),                       # 旅行クラス（例：ビジネス、エコノミー）
    StructField("is_recommended", BooleanType(), True),                 # 航空サービスを推薦するかどうか
    StructField("rating_overall", IntegerType(), True),                 # 全体評価
    StructField("rating_seat_comfort", IntegerType(), True),            # 座席の快適さの評価
    StructField("rating_cabin_staff_service", IntegerType(), True),     # 客室スタッフのサービスの評価
    StructField("rating_ground_service", IntegerType(), True),          # 地上サービスの評価
    StructField("rating_value_for_money", IntegerType(), True),         # 価格に対する価値の評価
    StructField("rating_food_beverages", IntegerType(), True),          # 飲食サービスの評価
    StructField("rating_ife", IntegerType(), True),                     # 機内エンタメの評価
    StructField("rating_wifi_connectivity", IntegerType(), True)        # 機内Wi-Fiの接続性の評価
])

# 構造体配列の作成
df_with_arrays = df.withColumn(
    "review_pos",
    array(
        struct(lit("食事と飲み物").alias("category"), col("pos_food_and_beverages_summary").alias("review")),
        struct(lit("機内エンターテイメント").alias("category"), col("pos_inflight_entertainment_summary").alias("review")),
        struct(lit("機内Wi-Fiと接続性").alias("category"), col("pos_wifi_connectivity_summary").alias("review")),
        struct(lit("価格に対する価値").alias("category"), col("pos_value_for_money_summary").alias("review")),
        struct(lit("地上サービス").alias("category"), col("pos_ground_service_summary").alias("review")),
        struct(lit("機内スタッフのサービス品質").alias("category"), col("pos_cabin_staff_service_summary").alias("review")),
        struct(lit("座席の快適さ").alias("category"), col("pos_seat_comfort_summary").alias("review"))
    )
).withColumn(
    "review_neg",
    array(
        struct(lit("食事と飲み物").alias("category"), col("neg_food_and_beverages_summary").alias("review")),
        struct(lit("機内エンターテイメント").alias("category"), col("neg_inflight_entertainment_summary").alias("review")),
        struct(lit("機内Wi-Fiと接続性").alias("category"), col("neg_wifi_connectivity_summary").alias("review")),
        struct(lit("価格に対する価値").alias("category"), col("neg_value_for_money_summary").alias("review")),
        struct(lit("地上サービス").alias("category"), col("neg_ground_service_summary").alias("review")),
        struct(lit("機内スタッフのサービス品質").alias("category"), col("neg_cabin_staff_service_summary").alias("review")),
        struct(lit("座席の快適さ").alias("category"), col("neg_seat_comfort_summary").alias("review"))
    )
)

# カラム順序（明示的に定義します）
main_columns = [
    "review_id",
    "user_id",
    "review_date",
    "review"
]

additional_columns = [
    "flight_id",
    "flight_date",
    "route_id",
    "seat_type",
    "is_recommended",
    "rating_overall",
    "rating_seat_comfort",
    "rating_cabin_staff_service",
    "rating_ground_service",
    "rating_value_for_money",
    "rating_food_beverages",
    "rating_ife",
    "rating_wifi_connectivity"
]

# ポジティブ展開処理
df_exploded_pos = df_with_arrays.select(
    *main_columns,
    explode("review_pos").alias("temp_data"),
    *additional_columns
).select(
    *main_columns,
    col("temp_data.category").alias("review_category"),
    col("temp_data.review").alias("review_details"),
    *additional_columns
).filter(col("review_details") != "-")

# ネガティブ展開処理
df_exploded_neg = df_with_arrays.select(
    *main_columns,
    explode("review_neg").alias("temp_data"),
    *additional_columns
).select(
    *main_columns,
    col("temp_data.category").alias("review_category"),
    col("temp_data.review").alias("review_details"),
    *additional_columns
).filter(col("review_details") != "-")

# データ型変換（全カラムに適用）
for field in schema.fields:
    col_name = field.name
    dtype = field.dataType
    df_exploded_pos = df_exploded_pos.withColumn(col_name, col(col_name).cast(dtype))
    df_exploded_neg = df_exploded_neg.withColumn(col_name, col(col_name).cast(dtype))

# 変換後のDataFrameを直接使用
df_exploded_pos_converted = df_exploded_pos
df_exploded_neg_converted = df_exploded_neg

# create tempview
df_exploded_pos_converted.createOrReplaceTempView("sv_reviews_exstracted_by_categories_pos")
df_exploded_neg_converted.createOrReplaceTempView("sv_reviews_exstracted_by_categories_neg")

# UNION
df = spark.sql('''
SELECT *, "positive" as sentiment FROM sv_reviews_exstracted_by_categories_pos
UNION ALL
SELECT *, "negative" as sentiment FROM sv_reviews_exstracted_by_categories_neg
''')

# TempViewテーブル作成
df.createOrReplaceTempView("v_reviews_exstracted_by_categories_all")

print(df.count())
print(df.columns)
display(df.limit(100))

In [0]:
from pyspark.sql.functions import col

'''
概要：llmが生成したサマリ文章に余計な文字列がも含まれていないかチェックする
処理：llmが生成したサマリから" -"が含まれるレコードを抽出
'''
# df_bad = df.filter(col("review_details") != "-")
df_bad = df.filter(col("review_details").contains(" -"))

print(df_bad.count())
print(df_bad.columns)
# display(df_bad.select("review_details").distinct())

## 4. ダッシュボードマート作成
作成テーブル：`gd_dashboard`  
ポジネガの画像を表示させるための事前処理を行います。  
ダッシュボードの表形式の設定で、ポジネガ（`sentiment`）の画像挿入のための動的パラメータとして活用します。

In [0]:
df = spark.sql(f"""
SELECT
  '{HOST_URL}' AS HOST_URL,              -- ホストURL（ダッシュボードの画像挿入のための動的パラメータとして活用します） 
  '{MY_CATALOG}' AS MY_CATALOG,          -- カタログ名（ダッシュボードの画像挿入のための動的パラメータとして活用します）
  '{MY_SCHEMA}' AS MY_SCHEMA,            -- スキーマ名（ダッシュボードの画像挿入のための動的パラメータとして活用します）
  '{MY_VOLUME_IMG}' AS MY_VOLUME_IMG,
  review_id,
  user_id,
  review_date,
  review,
  review_category,
  review_details,
  flight_id,
  flight_date,
  route_id,
  seat_type,
  is_recommended,
  rating_overall,
  rating_seat_comfort,
  rating_cabin_staff_service,
  rating_ground_service,
  rating_value_for_money,
  rating_food_beverages,
  rating_ife,
  rating_wifi_connectivity,
  sentiment
FROM
  v_reviews_exstracted_by_categories_all
""")

# write table
df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.gd_dashboard")

print(df.count())
print(df.columns)
display(df.limit(100))

In [0]:
'''変数定義'''
TABLE_PATH = f'{MY_CATALOG}.{MY_SCHEMA}.gd_dashboard'                 # テーブルパス
PK_CONSTRAINT_NAME = f'pk_gd_dashboard'                               # 主キー

'''NOT NULL制約の追加'''
columns_to_set_not_null = ['review_id',]

for column in columns_to_set_not_null:
    spark.sql(f"""
    ALTER TABLE {TABLE_PATH}
    ALTER COLUMN {column} SET NOT NULL;
    """)

'''主キー設定'''
spark.sql(f'''
ALTER TABLE {TABLE_PATH} DROP CONSTRAINT IF EXISTS {PK_CONSTRAINT_NAME};
''')

spark.sql(f'''
ALTER TABLE {TABLE_PATH}
ADD CONSTRAINT {PK_CONSTRAINT_NAME} PRIMARY KEY (review_id);
''')

# # チェック
# display(
#     spark.sql(f'''
#     DESCRIBE EXTENDED {TABLE_PATH}
#     '''))


'''認定済みタグの追加'''
certified_tag = 'system.Certified'

try:
    spark.sql(f"ALTER TABLE {TABLE_PATH} SET TAGS ('{certified_tag}')")
    print(f"認定済みタグ '{certified_tag}' の追加が完了しました。")

except Exception as e:
    print(f"認定済みタグ '{certified_tag}' の追加中にエラーが発生しました: {str(e)}")
    print("このエラーはタグ機能に対応していないワークスペースで実行した場合に発生する可能性があります。")

In [0]:
# テーブルコメント
comment = """
テーブル名：`gd_dashboard / VoC分析ダッシュボードマート`  
説明：航空サービスの顧客レビューを用いたVoC分析ダッシュボードが参照するマートテーブルです。LLMを用いて、顧客レビューから論点抽出・要約をしています。LLMが判定している論点は次のとおり。  
- カテゴリ
  - 座席の快適さ
  - 客室スタッフサービス
  - 地上サービス
  - 価格に対する満足度
  - 飲食サービス
  - 機内エンターテイメント
  - 機内Wi-Fiの接続性
- 感情
  - ポジティブ
  - ネガティブ
"""
spark.sql(f'COMMENT ON TABLE {TABLE_PATH} IS "{comment}"')

# カラムコメント
column_comments = {
    "HOST_URL": "ホストURL、ダッシュボードの画像挿入のための動的パラメータとして活用",
    "MY_CATALOG": "カタログ名、ダッシュボードの画像挿入のための動的パラメータとして活用",
    "MY_SCHEMA": "スキーマ名、ダッシュボードの画像挿入のための動的パラメータとして活用",
    "MY_VOLUME_IMG": "ボリューム名、ダッシュボードの画像挿入のための動的パラメータとして活用",
    "review_id": "レビューID、主キー、例）1",
    "user_id": "会員ID、例）1",
    "review_date": "レビュー投稿日、YYYY-MM-DDフォーマット",
    "review": "レビュー本文",
    "review_category": "レビューカテゴリ、例）座席の快適さ",
    "review_details": "カテゴリ別のレビュー、LLMが抽出・要約した文章、例）'機内の快適さが素晴らしく、料金に見合った価値を感じました。'",
    "flight_id": "便名（機材＋日付で一意）、主キー、例）JL006",
    "flight_date":"出発日、YYYY-MM-DDフォーマット",
    "route_id":"区間、例）NYC-NRT",
    "seat_type": "旅行クラス、例）'Premium Economy', 'Economy Class', 'First Class', 'Business Class'",
    "is_recommended": "推奨意向、例）True, False",
    "rating_overall": "総合評価、5段階評価、例）4",
    "rating_seat_comfort": "座席の快適さの評価、5段階評価、例）4",
    "rating_cabin_staff_service": "客室スタッフのサービスの評価、5段階評価、例）4",
    "rating_ground_service": "地上サービスの評価、5段階評価、例）4",
    "rating_value_for_money": "価格に対する価値の評価、5段階評価、例）4",
    "rating_food_beverages": "飲食サービスの評価、5段階評価、例）4",
    "rating_ife": "機内エンタメの評価、5段階評価、例）4",
    "rating_wifi_connectivity": "機内Wi-Fiの接続性の評価、5段階評価、例）4",
    "sentiment": "ポジネガ、例）'positive', 'negative'"
}

for column, comment in column_comments.items():
    escaped_comment = comment.replace("'", "\\'")
    sql_query = f"ALTER TABLE {TABLE_PATH} ALTER COLUMN {column} COMMENT '{escaped_comment}'"
    spark.sql(sql_query)